<a href="https://colab.research.google.com/github/with2say/layout2inspection/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/code

/content/drive/MyDrive/code


In [ ]:
!git clone https://github.com/with2say/layout2inspection.git

Cloning into 'layout2inspection'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 5), reused 10 (delta 3), pack-reused 0
Unpacking objects: 100% (18/18), 7.31 KiB | 1.83 MiB/s, done.


In [ ]:
!git config --global user.email 'with2say@gmail.com'
!git config --global user.name 'with2say'

In [ ]:
!git add main.ipynb
!git commit -m "Add main module"
!git push

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 21.1 MB/s eta 0:00:00


In [ ]:
!cd /content/drive/MyDrive/code/layout2inspection
!dir
#!git remote add origin https://github.com/with2say/layout2inspection.git

layout2inspection  layout2inspection_old


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from dataset import *
from model import *


def get_predictions(trainer, model, data_module):
    model.eval()
    model.to('cpu')
    y_true = []
    y_pred = []

    predictions = trainer.predict(model, datamodule=data_module)
    for pred, y in predictions:
        y_true.extend(y.view(-1).tolist())
        y_pred.extend(pred.view(-1).tolist())

    return y_true, y_pred


def plot_true_vs_predicted(y_true, y_pred):
    sns.scatterplot(x=y_true, y=y_pred, alpha=0.5)
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title('True vs Predicted Values')
    plt.tight_layout()
    plt.show()


if __name__ == '__main__':

    # input params
    num_samples = 1000
    sequence_range = [1, 3]
    polygon_range = [3, 5]
    num_position = 2

    # hyper params
    data, targets = generate_data(num_samples, sequence_range, polygon_range)
    print('target mse:', np.var(targets))

    data_module = PolygonAreaDataModule(data, targets, batch_size=256, val_split=0.05, test_split=0.05, num_workers=4)
    print(np.shape(targets))

    # Create the model
    layer = Polygons2Area(d_model=64,
                          nhead=16,
                          num_layers=16,
                          num_position=num_position,
                          num_polygons=np.max(polygon_range),
                          dropout=0.01)

    # d_model = num_position
    # nhead = 2
    # num_layers = 2
    # dim_feedforward = 4
    # layer = HierarchicalTransformer(d_model=num_position,
    #                                 nhead=nhead,
    #                                 num_layers=num_layers,
    #                                 dim_feedforward=dim_feedforward)

    model = PolygonRegressor(layer)

    trainer = pl.Trainer(max_epochs=3)
    trainer.fit(model, data_module)
    trainer.validate(model, datamodule=data_module)

    y_true, y_pred = get_predictions(trainer, model, data_module)
    plot_true_vs_predicted(y_true, y_pred)




/content/drive/MyDrive/code/layout2inspection/dataset.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data), np.array(targets).reshape(-1, 1).astype(np.float32)


target mse: 0.016796876
(1000, 1)


/usr/local/lib/python3.9/dist-packages/lightning_fabric/accelerators/cuda.py:235: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type          | Params
----------------------------------------
0 | layer | Polygons2Area | 850 K 
----------------------------------------
850 K     Trainable params
0         Non-trainable params
850 K     Total params
3.402     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


TypeError: ignored